In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from keras.layers import Dense, Input
from keras import Sequential

Using TensorFlow backend.


In [2]:
trainDf = pd.read_csv('/import/scratch/mdl31/ctrdata/train')

In [3]:
# One-hotencode the categorical variables:
hourSeries = trainDf['hour']
idSeries = trainDf['id']
ySeries = trainDf['click']

In [4]:
trainDf = trainDf.drop('hour', axis = 1)
trainDf = trainDf.drop('id', axis = 1)
trainDf = trainDf.drop('click', axis = 1)


# Removing these as there are too many categories
# TODO could try to group into broad categories, e.g. take top 5 commonest categroies, and put the rest under 'other' token
trainDf = trainDf.drop('device_id', axis = 1)
trainDf = trainDf.drop('device_ip', axis = 1)
trainDf = trainDf.drop('device_model', axis = 1)
trainDf = trainDf.drop('site_id', axis = 1)
trainDf = trainDf.drop('site_domain', axis = 1)
trainDf = trainDf.drop('app_id', axis = 1)
trainDf = trainDf.drop('app_domain', axis = 1)
trainDf = trainDf.drop('C14', axis = 1)
trainDf = trainDf.drop('C17', axis = 1)
trainDf = trainDf.drop('C19', axis = 1) # only 68 values so might want
trainDf = trainDf.drop('C20', axis = 1)

In [ ]:
combs = 1
for col in trainDf.columns:
    print(col)
    length = len(trainDf[col].value_counts())
    combs *= length
    print(length)

In [6]:
trainDf = pd.get_dummies(trainDf, columns = trainDf.columns, drop_first=True)

In [7]:
len(trainDf.columns)

156

In [8]:
trainDf['hour'] = hourSeries
#trainDf['id'] = idSeries
del hourSeries


In [9]:
# Create validation dataset
x_train, x_val, y_train, y_val = train_test_split(trainDf, ySeries, stratify = ySeries)
del trainDf

In [ ]:
'''# Now to start building a model
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(max_iter=2, loss='hinge')
model.fit(x_train,y_train)
y_train_pred = model.pred(x_train)
accuracy_score(y_train,y_train_pred)'''

In [10]:
nnModel = Sequential()
nnModel.add(Dense(1024, input_shape=(len(x_train.columns),), activation='relu'))
nnModel.add(Dense(512, activation='relu'))
nnModel.add(Dense(256, activation='relu'))
nnModel.add(Dense(1,   activation='sigmoid'))
nnModel.compile(optimizer='adam', metrics=['accuracy'],loss='binary_crossentropy')
nnModel.summary()


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              161792    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 818,177
Trainable params: 818,177
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nnModel.fit(x_train,y_train, batch_size = 256, epochs = 10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 3198720/30321725 [==>...........................] - ETA: 17:55 - loss: 2.7397 - acc: 0.8300